# Module 2: Combinational Logic

Adapted for UC Berkeley ELENG 194/294 from Chisel Bootcamp by ELENG 194/294 staff

In this section you will see how to use Chisel components to implement combinational logic.
We will demonstrate how three of the basic Chisel types: `UInt `- unsigned integer; `SInt` - signed integer, and `Bool` - true or false may be connected and operated upon.
Notice how all Chisel variables are declared as Scala `val`s.
Never use a Scala `var` for a hardware construct, since the construct itself may never change once defined; only its value may change when running the hardware.
Wires may be used for parameterized types.


In [2]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/Users/williamlyh/Desktop/Berkeley/Spring2024/ELENG194/bootcamp/tapeout-chisel-bootcamp/source/load-ivy.sc"

In [3]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test


---
# Common Operators
Now that you understand how `Module`s are constructed, let's make some hardware! Take a look at the empty module below.

In [4]:
class MyModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
}

defined class MyModule

We've called our class `MyModule`, and it extends `Module`. This means it gets mapped to a hardware module in Verilog. Our `MyModule` module has one input and one output. The input is a 4-bit unsigned integer (`UInt`), and so is the output. 

<span style="color:blue">**Example: Scala and Chisel Operators Look the Same**</span><br>
Let's look at different operations we can perform on data.

In [5]:
class MyModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })

  val two  = 1 + 1
  println(two)
  val utwo = 1.U + 1.U
  println(utwo)
  
  io.out := io.in
}
println(getVerilog(new MyModule))

Elaborating design...
2
UInt<1>(OpResult in MyModule)
Done elaborating.
module MyModule(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out
);
  assign io_out = io_in; // @[cmd4.sc 12:10]
endmodule


defined class MyModule

We create two `val`s. The first adds two Scala `Int`s, so `println` prints out the integer 2. The second `val` adds two *Chisel* `UInt`s together, so `println` sees this as a hardware node and prints out the type name and pointer (`chisel3.core.UInt@d`). Note that `1.U` is a type cast from a Scala `Int` (1) to a Chisel `UInt` literal.

We need to drive the output to something, so we just connect it to the input for now, as with the passthrough module in the previous tutorial.

<span style="color:blue">**Example: Incompatible Operation**</span><br>
What happens if we add a Chisel `1.U` to the literal `1`? These types are incompatible, as the former is a hardware wire of value 1, while the latter is a Scala value of 1. So Chisel will give a type mismatch error.

In [ ]:
class MyModuleTwo extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })

  val twotwo = 1.U + 1
  println(twotwo)
  
  io.out := io.in
}
println(getVerilog(new MyModuleTwo))

It's important to remember the distinction between types when performing operations. Scala is a strongly typed language, so any type casting must be explicit.

<span style="color:blue">**Example: More Chisel Operators**</span><br>
Other common operations are subtraction and multiplication. These are handled on unsigned integers as expected. Let's  see these in action. We show the Verilog, though there's some underlying Chisel features that obfuscate the simple code we would expect.

In [ ]:
class MyOperators extends Module {
  val io = IO(new Bundle {
    val in      = Input(UInt(4.W))
    val out_add = Output(UInt(4.W))
    val out_sub = Output(UInt(4.W))
    val out_mul = Output(UInt(4.W))
  })

  io.out_add := 1.U + 4.U
  io.out_sub := 2.U - 1.U
  io.out_mul := 4.U * 2.U
}
println(getVerilog(new MyOperators))

And here's a sample tester for the above operations. Instead of using an anonymous tester class like in the previous tutorial, we'll create an explicit tester class. This is just an alternative way of writing a tester.

In [ ]:
test(new MyOperators) {c =>
  c.io.out_add.expect(5.U)
  c.io.out_sub.expect(1.U)
  c.io.out_mul.expect(8.U)
}
println("SUCCESS!!")

<span style="color:blue">**Example: Mux and Concatenation**</span><br>
In addition to addition, subtraction, and multplication, Chisel has mux and concatenation operators. These are shown below. The `Mux` operates like a traditional ternary operator, with the order (select, value if true, value if false). Note that `true.B` and `false.B` are the preferred ways to create Chisel Bool literals. The `Cat` ordering is MSB then LSB (where B refers to bit or bits), and only takes two arguments. Concatenating more than two values requires multiple `Cat` calls or advanced Chisel and Scala features covered in later sections.

In [ ]:
class MyOperatorsTwo extends Module {
  val io = IO(new Bundle {
    val in      = Input(UInt(4.W))
    val out_mux = Output(UInt(4.W))
    val out_cat = Output(UInt(4.W))
  })

  val s = true.B
  io.out_mux := Mux(s, 3.U, 0.U) // should return 3.U, since s is true
  io.out_cat := Cat(2.U, 1.U)    // concatenates 2 (b10) with 1 (b1) to give 5 (101)
}

println(getVerilog(new MyOperatorsTwo))

test(new MyOperatorsTwo) { c =>
  c.io.out_mux.expect(3.U)
  c.io.out_cat.expect(5.U)
}
println("SUCCESS!!")

Notice how the Verilog contains constants instead of actual mux or concatenation logic. This is because FIRRTL transformations have simplified the circuit, eliminating obvious logic. 

For a more complete list of Chisel operators, see the [Chisel cheatsheet](https://github.com/freechipsproject/chisel-cheatsheet/releases/latest/download/chisel_cheatsheet.pdf). For the most complete list of operators and their implementation details, look through the [Chisel API](https://chisel-lang.org/api/latest/).

---
# Exercises
To complete these exercises, you may need to look through the [Chisel cheatsheet](https://github.com/freechipsproject/chisel-cheatsheet/releases/latest/download/chisel_cheatsheet.pdf).

<span style="color:red">**Exercise: MAC**</span><br>
Create a Chisel module that implements the multiply accumulate function, `(A*B)+C`, and passes the testbench.

In [ ]:
class MAC extends Module {
  val io = IO(new Bundle {
    val in_a = Input(UInt(4.W))
    val in_b = Input(UInt(4.W))
    val in_c = Input(UInt(4.W))
    val out  = Output(UInt(8.W))
  })

  ???
}

test(new MAC) { c =>
  val cycles = 100
  import scala.util.Random
  for (i <- 0 until cycles) {
    val in_a = Random.nextInt(16)
    val in_b = Random.nextInt(16)
    val in_c = Random.nextInt(16)
    c.io.in_a.poke(in_a.U)
    c.io.in_b.poke(in_b.U)
    c.io.in_c.poke(in_c.U)
    c.io.out.expect((in_a * in_b + in_c).U)
  }
}
println("SUCCESS!!")

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
class MAC extends Module {
  val io = IO(new Bundle {
    val in_a = Input(UInt(4.W))
    val in_b = Input(UInt(4.W))
    val in_c = Input(UInt(4.W))
    val out  = Output(UInt(8.W))
  })

  io.out := (io.in_a * io.in_b) + io.in_c
}
</pre></article></div></section></div>